In [1]:
from pyswip.prolog import Prolog
from pyswip import registerForeign, Functor, call, Variable
import tempfile
import os
import sys

In [2]:
### Readable Location names
dict_locations = {
    "museo_evita" : "Museo Evita: A museum all about Eva Perón, Argentina's famous first lady.",
    "science_museum": "Centro Cultural de la Ciencia: A hands-on science museum.",
    "fine_arts": "Museo Nacional de Bellas Artes: Visit Argentina's largest fine arts museum.",
    "borges_center": "Borges Cultural Center: A historic landmark and exhibit dedicated to Argentinian writer, Jorge Luis Borges.",
    "silvori_museum": "Museo Sívori: A small art museum featuring Argentinian artists.",
    "malba": "Museo de Arte Latinoamericano de Buenos Aires (MALBA): The famous museum dedicated to latin-american art.",
    "museo_modern": "Museo Moderno: Visit Buenos Aires' museum for modern art and movements.",
    "library": "Biblioteca Nacional Mariano Moreno: Visit Argentina's beautiful national library, located at the heart of a park.",
    "recoletta_cemetery": "Cementerio de la Recoleta: The recoleta cemetery contains graves for some of the most famous Argentinians in history.",
    "basilica": "Basilica Maria Auxiliadora y San Carlos: Buenos Aires' most beautiful church.",
    "jazz_backroom": "Backroom Bar: Enjoy live jazz at backroom, the bar and cafe and library.",
    "tango": "MOVAQ - Aquelarre en Movimiento: Learn to tango in this Minervan-approved dance class.",
    "teatro_colon": "Teatro Colón: Book tickets or visit Buenos Aires' opera house.",
    "botanical_gardens": "Jardín Botánico Carlos Thays: Run through Buenos Aires' many gardens and parks at the botanical gardens.",
    "football_centro_garrigos": "Centro Garrigós: Visit centro garrigos and play Argentina's favourite sport on their free field!",
    "ecological_reserve": "Reserva Ecológica Costanera Sur: Get out of the city and bike through Buenos Aires' Ecological Reserve.",
    "centenario_park": "Parque Centenario: Rent a bike to travel around centenario park and enjoy its lake!",
    "rock_climb": "Bien Alto Escuela de Montaña: Learn to rock climb.",
    "peru_beach":"Peru Beach: Enjoy one of the many sports available at Peru Beach sportsclub.",
    "veganious":"Veganious: Cheap vegan eats!",
    "toque_perfecto":"Toque Perfecto: Vegetarian buffet with all the vegetables you can dream of.",
    "cang_tin":"Cang Tin: A vietnamese restaurant with fun decor.",
    "saigon_noodle":"Saigon Noodle Bar: Best vietnamese food in Buenos Aires (according to non-vietnamese Minervans)",
    "chui": "Chui: The best restaurant in the city (highly recommend the mushroom pâté.)",
    "mudra": "Mudra: Vegetarian high dining.",
    "tandoor": "Tandoor: One of the few Indian restaurants in Buenos Aires.",
    "chori": "Chori: Grab Argentina's favourite cheap lunch, a choripan!",
    "sazon_cuyagua": "El Sazón De Cuyagua: The best venezuelan empanadas in the city (plus more cheap venezuelan meals).",
    "koi_dumplings": "Koi Dumplings: ",
    "kefi_greek": "Kefi: One of the few greek restaurants in Buenos Aires.",
    "concina_yovita": "La Concina de Yovita: Expect massive portions of delicious Peruvian dishes.",
    "mr_ho_korean": "Mr.Ho's: Excellent Korean food for when you are craving spice.",
    "don_julio": "Don Julio: Come here for world famous Argentinian steak.",
    "sagardi_argentina": "Sagardi: High dining meets Argentian barbecue."
}

## Interface Scaffold

In [3]:
def consult_expert_system(debug=False):
    """
    Consults a knowledge base of tourist locations in Buenos Aires and asks a series of
    diagnostic questions to determine recommended activities. Uses Prolog's predicate logic
    to analyse previous answers and determine which questions need to be asked for resolution.

    Inputs
    -------
    debug: if true, prints all known values and attributes when running the expert system

    Outputs: None
    """

    prolog = Prolog()
    prolog.consult("./buenos_aires-kb.pl")

    # pull out Prolog built-ins into the Python name-space
    retractall = Functor("retractall")
    known = Functor("known", 3)
    member = Functor('member', 2)

    # maps basic attributes to natural-language questions
    question_dict = {
        'budget': "Should the activity be free? (yes/no)",
        'experience': "Which of the following experiences would you like to have? (Input the associated number)",
        'time_required': "How long should the activity take? (Short is less than 2 hours)",
        'distance': "Do you want to do something nearby or far away (Input the associated number)",
        'price_point': "Roughly how much do you want to pay for your food? (Input the associated number",
        'culture_type': "Would you like to go to a museum? (yes/no)",
        'museum_type': "What type of museum would you like to visit? (Input the associated number)",
        'diet': "Are you vegetarian (yes/no)"
    }

    def read_py(A,V,Y):
        """
        Asks the user whether a specified value V of an attribute A is true,
        and binds the response to a Prolog variable.

         Inputs
        --------
        A: the specified attribute
        V: the associated value
        Y: a variable with which the user response is unified

        Output
        ------
        success/failure: returns true as long as Y is a variable, and false otherwise; maintains Prolog execution flow

        """
        if isinstance(Y, Variable):
            response = input(question_dict[str(A)])

            # handle variants on "Yes", allowing for whitespace and capitalisation
            processed_response = response.lower().strip()
            print(f"Question: {question_dict[str(A)]}, Response: {processed_response}")
            Y.unify(processed_response)
            return True
        else:
            return False

    def read_menu_py(A, X, Menu):
        """
        Asks the user to specify which value V of several possible menu options in Menu
        for a given attribute A they prefer. Unifies the result with X, which is a variable
        that is then passed into Prolog's predicate logic system.

        Inputs
        -----
        A: the specified attribute
        X: the variable used to encode user responses
        Menu: the list of possible values for the attribute

        """
        if isinstance(X, Variable):
            menu_dict = {str(i+1): str(Menu[i]) for i in range(len(Menu))}
            options = '\n'.join([f'{i}. {j}' for i, j in menu_dict.items()])
            response = input(f"{question_dict[str(A)]}: {options}")

            if str(response) in [str(i+1) for i in range(len(Menu))]:
                print(f"Question: {question_dict[str(A)]}, Response: {menu_dict.get(response)} [{response}]")
            
            # fail gracefully on incorrect input and try again
            X.unify(menu_dict.get(response, str(response)))

            
            return True
        else:
            return False
        
    def ask_menu_again_py(Z):
        """
        Printing function for cases where Prolog does not recognise the input to menuask
        as a legal value

        Inputs
        --------
        Z: the user input

        Outputs
        --------
        Returns true so that Prolog continues execution

        """
        print(f"{Z} is not a legal value, try again.")
        return True

    # registering functions for use in prolog
    read_py.arity=3
    ask_menu_again_py.arity=1
    read_menu_py.arity=3

    registerForeign(read_py)
    registerForeign(read_menu_py)
    registerForeign(ask_menu_again_py)

    call(retractall(known))
    destination = [s for s in prolog.query("findall(Location, recommended(Location), List).")]
    locations = list(set(destination[0]['List']))

    if debug:
        # prints all known attributes and values
        known_attrs = [s for s in prolog.query("findall(X, known(yes, X, _), List).")]
        known_vals = [s for s in prolog.query("findall(Y, known(yes, _, Y), List).")]
        print("\n Known Attributes:", known_attrs)
        print("\n Known Values:", known_vals)

    # intelligently print final recommendation(s) besed on how many there are
    if not locations:
        print("unknown")
    elif len(locations) == 1:
        print(f"Your recommended destination is: \n {dict_locations[locations[0]]}")
    else:
        print(f"Some recommended destinations are: \n- " + '\n- '.join([dict_locations[i] for i in locations]))

## Test Cases

### Test Case One
- Experience: Cultural
- Time Required: Long (2)
- Distance: Far (2)
- Budget: Free
- Museum?: Yes
- Art or Cultural Museum: Art

 -> "Museo Nacional de Bellas Artes: Visit Argentina's largest fine arts museum."

### Test Case Two
- Experience: Physical Activity
- Time Required: Short (1)
- Budget: Paid

 -> "Parque Centenario: Bike around centenario park and enjoy its lake!"

### Test Three
- Experience: Culinary
- Distance: Close (1)
- Price: High (3)
- Vegetarian: No

 -> "Don Julio: Come here for world famous Argentinian steak."

In [5]:
# test case one
consult_expert_system()

Question: Which of the following experiences would you like to have? (Input the associated number), Response: cultural [1]
Question: How long should the activity take? (Short is less than 2 hours), Response: long [2]
Question: Do you want to do something nearby or far away (Input the associated number), Response: far [2]
Question: Should the activity be free? (yes/no), Response: yes
Question: Would you like to go to a museum? (yes/no), Response: yes
Question: What type of museum would you like to visit? (Input the associated number), Response: art [1]
Your recommended destination is: 
 Museo Nacional de Bellas Artes: Visit Argentina's largest fine arts museum.


In [6]:
# test case two
consult_expert_system()

Question: Which of the following experiences would you like to have? (Input the associated number), Response: physical_activity [2]
Question: How long should the activity take? (Short is less than 2 hours), Response: short [1]
Question: Should the activity be free? (yes/no), Response: no
Your recommended destination is: 
 Parque Centenario: Rent a bike to travel around centenario park and enjoy its lake!


In [7]:
# test case three
consult_expert_system()

Question: Which of the following experiences would you like to have? (Input the associated number), Response: culinary [3]
Question: Do you want to do something nearby or far away (Input the associated number), Response: close [1]
Question: Roughly how much do you want to pay for your food? (Input the associated number, Response: high [3]
Question: Are you vegetarian (yes/no), Response: no
Your recommended destination is: 
 Don Julio: Come here for world famous Argentinian steak.
